In [1]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O /tmp/cats_and_dogs_filtered.zip

--2020-09-05 10:08:55--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.122.128, 172.217.13.80, 172.217.13.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.122.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   252MB/s    in 0.3s    

2020-09-05 10:08:56 (252 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [2]:
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

In [3]:
from pathlib import Path

In [4]:
base_dir = Path('/tmp/cats_and_dogs_filtered')
train_dir = base_dir / Path('train')
validation_dir = base_dir / Path('validation')

In [5]:
train_cats_dir = train_dir / Path('cats')
train_dogs_dir = train_dir / Path('dogs')

In [6]:
validation_cats_dir = validation_dir / Path('cats')
validation_dogs_dir = validation_dir / Path('dogs')

In [7]:
train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

In [8]:
print(train_cat_fnames[:10])
print(train_dog_fnames[:10])

['cat.127.jpg', 'cat.126.jpg', 'cat.125.jpg', 'cat.124.jpg', 'cat.123.jpg', 'cat.122.jpg', 'cat.121.jpg', 'cat.120.jpg', 'cat.119.jpg', 'cat.118.jpg']
['dog.127.jpg', 'dog.126.jpg', 'dog.125.jpg', 'dog.124.jpg', 'dog.123.jpg', 'dog.122.jpg', 'dog.121.jpg', 'dog.120.jpg', 'dog.119.jpg', 'dog.118.jpg']


In [9]:
print('Total training cat images:', len(os.listdir(train_cats_dir)))
print('Total training dog images:', len(os.listdir(train_dogs_dir)))

Total training cat images: 1000
Total training dog images: 1000


In [10]:
print('Total validation cat images:', len(os.listdir(validation_cats_dir)))
print('Total validation dog images:', len(os.listdir(validation_dogs_dir)))

Total validation cat images: 500
Total validation dog images: 500


In [11]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# graph parameters
n_rows = 4
n_columns = 4

pic_index = 0

In [12]:
import tensorflow as tf

In [13]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    # flatten to feed into DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [15]:
from tensorflow.keras.optimizers import RMSprop

In [16]:
model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [18]:
train_datagen = ImageDataGenerator( rescale=1.0/255.)
validation_datagen = ImageDataGenerator( rescale=1.0/255.)

In [19]:
# Flow images in batches of 20 using datagen generator

train_generator = train_datagen.flow_from_directory(train_dir, batch_size=20, class_mode='binary',target_size=(150,150))
validation_generator = validation_datagen.flow_from_directory(validation_dir, batch_size=20, class_mode='binary',target_size=(150,150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [22]:
history = model.fit(train_generator, validation_data=validation_generator, steps_per_epoch=100, epochs=15, validation_steps=50, verbose=2)

Epoch 1/15
100/100 - 4s - loss: 0.0526 - accuracy: 0.9940 - val_loss: 2.9729 - val_accuracy: 0.7030
Epoch 2/15
100/100 - 4s - loss: 1.2440e-04 - accuracy: 1.0000 - val_loss: 3.5880 - val_accuracy: 0.7060
Epoch 3/15
100/100 - 4s - loss: 0.0790 - accuracy: 0.9935 - val_loss: 3.7956 - val_accuracy: 0.7010
Epoch 4/15
100/100 - 4s - loss: 1.2814e-05 - accuracy: 1.0000 - val_loss: 3.9388 - val_accuracy: 0.7050
Epoch 5/15
100/100 - 4s - loss: 0.1411 - accuracy: 0.9895 - val_loss: 3.4824 - val_accuracy: 0.7100
Epoch 6/15
100/100 - 4s - loss: 9.5796e-06 - accuracy: 1.0000 - val_loss: 3.7292 - val_accuracy: 0.7100
Epoch 7/15
100/100 - 4s - loss: 0.0850 - accuracy: 0.9950 - val_loss: 4.6841 - val_accuracy: 0.7170
Epoch 8/15
100/100 - 4s - loss: 0.0188 - accuracy: 0.9965 - val_loss: 3.8081 - val_accuracy: 0.7250
Epoch 9/15
100/100 - 4s - loss: 0.0327 - accuracy: 0.9945 - val_loss: 3.9104 - val_accuracy: 0.6890
Epoch 10/15
100/100 - 4s - loss: 0.0108 - accuracy: 0.9975 - val_loss: 4.3165 - val_accu